## Interface for the Uusimaa map

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, IntSlider, interactive
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import sklearn
from sklearn.linear_model import LinearRegression
import json
from pandas.io.json import json_normalize
import math
from sympy import geometry
from pyproj import Proj, transform
import matplotlib.patches as mpatches
import os

In [2]:
#Load the data
s=['Espoo','Helsinki','Hyvinkää','Järvenpää','Karkkila',
          'Kauniainen','Kerava','Kirkkonummi','Lohja','Mäntsälä',
          'Nurmijärvi','Pornainen','Sipoo','Siuntio','Tuusula',
          'Vantaa','Vihti','Lapinjärvi','Loviisa','Askola',
          'Myrskylä','Porvoo','Pukkila','Hanko','Inkoo','Raasepori']
years=[2005 + i for i in range(13)]
r={}
for i in range(13):    
    h={}
    year=2005+i
    print(year)
    accident=pd.read_excel('Data/{}/UusimaaAccident.xlsx'.format(year))
    data=accident[['accidentId','dead','injr','munidesc']]
    level=[3*data['dead'][i] + data['injr'][i] for i in range(len(data))]
    data=pd.merge(data,pd.Series(level,name='severlvl'),left_index=True,right_index=True)
    for mun in s:
        v=dict(data[data['munidesc']==mun]['severlvl'].value_counts())
        #Here we set the level of severity for each municipality of Uusimaa.
        h[mun]=sum([(i+1)*v[i] for i in v.keys()])/sum(v.values())
    r[year]=h
values=r
print('DATA LOADED')

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
DATA LOADED


In [3]:
#Uusimaa on the map
with open('Data/Uusimaa.txt') as u:
    u=open('Data/Uusimaa.txt')
    Uusimaa=json.loads(u.read())
df=json_normalize(Uusimaa['features'])
c=pd.Series(df['geometry.coordinates'].fillna(df['geometry.geometries']),name='Shapes')
df=df.join(c)
#print(df['properties.name'])
df['properties.name'][5]='Hyvinkää'
df['properties.name'][7]='Järvenpää'
df['properties.name'][12]='Lapinjärvi'
df['properties.name'][14]='Myrskylä'
df['properties.name'][15]='Mäntsälä'
df['properties.name'][16]='Nurmijärvi'
#print(df['properties.name'])
#k=0
#inProj = Proj(init='epsg:4258')
#outProj = Proj(init='epsg:3067')
print('MAP DRAWN')

MAP DRAWN


In [4]:
def plot(year):
    matplotlib.rcParams['figure.figsize'] = [12.8, 7.2]
    mun=[]
    clist=['cyan','deepskyblue','dodgerblue','royalblue','blue','darkblue']
    heat=values[year]
    c='white'
    for i in range(len(df)):
        t=df['Shapes'][i]
        mun.append(df['properties.name'][i])
        if heat[df['properties.name'][i]]<=1:
            c=clist[0]
        if (heat[df['properties.name'][i]]>1) & (heat[df['properties.name'][i]]<=1.25) :
            c=clist[1]
        if (heat[df['properties.name'][i]]>1.25) & (heat[df['properties.name'][i]]<=1.50) :
            c=clist[2]
        if (heat[df['properties.name'][i]]>1.50) & (heat[df['properties.name'][i]]<=1.75) :
            c=clist[3]
        if (heat[df['properties.name'][i]]>1.75) & (heat[df['properties.name'][i]]<=2.00) :
            c=clist[4]
        if heat[df['properties.name'][i]]>2.0:
            c=clist[5]
            
        if len(t)==1:
            x=[t[0][l][0] for l in range(len(t[0]))]
            y=[t[0][l][1] for l in range(len(t[0]))]
    #        x,y=transform(inProj,outProj,x,y)
            plt.fill(x,y,c)
            plt.plot(x,y,c='k',linewidth=0.5)
                
                
        else:
            for s in t:
                x=[s['coordinates'][0][l][0] for l in range(len(s['coordinates'][0]))]
                y=[s['coordinates'][0][l][1] for l in range(len(s['coordinates'][0]))]
                #x,y=transform(inProj,outProj,x,y)
                plt.fill(x,y,c)
                plt.plot(x,y,c='k',linewidth=0.5)
                p1=mpatches.Patch(color=clist[0], label='<=1.0')
                p2=mpatches.Patch(color=clist[1], label='1.01-1.25')
                p3=mpatches.Patch(color=clist[2], label='1.26-1.50')
                p4=mpatches.Patch(color=clist[3], label='1.51-1.75')
                p5=mpatches.Patch(color=clist[4], label='1.76-2.00')
                p6=mpatches.Patch(color=clist[5], label='>2.0')
    patches=[p1,p2,p3,p4,p5,p6]
    plt.legend(bbox_to_anchor=(1.05,1.00),handles=patches)
    plt.title('Relative severity of traffic accidents in the municipalities of Uusimaa during the year {}'.format(year))

In [5]:
print('SELECT THE YEAR')
interact_manual(plot,year=IntSlider(min=2005, max=2017, step=1))

SELECT THE YEAR


interactive(children=(IntSlider(value=2005, description='year', max=2017, min=2005), Button(description='Run I…

<function __main__.plot(year)>

In [6]:
#!jupyter nbconvert --to html heatInter2.ipynb

[NbConvertApp] Converting notebook heatInter2.ipynb to html
[NbConvertApp] Writing 303597 bytes to heatInter2.html
